In [1]:
import numpy as np
import pandas as pd
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import Convolution1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.merge import Concatenate
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


Using TensorFlow backend.


In [ ]:
# read in high and low expression one_hot files
#high_exp = pd.concat(
#    [pd.read_csv("../c_nyambr/high_exp_one_hot_{i}.csv", index_col=0)
#     for i in range(1, 5)])
#low_exp = pd.concat(
#    [pd.read_csv("../c_nyambr/low_exp_one_hot_{i}.csv", index_col=0)
#     for i in range(1, 5)])

In [2]:
high_exp = pd.concat([pd.read_csv('high_exp_one_hot_1.csv',index_col=0),
                      pd.read_csv('high_exp_one_hot_2.csv',index_col=0),
                      pd.read_csv('high_exp_one_hot_3.csv',index_col=0),
                      pd.read_csv('high_exp_one_hot_4.csv',index_col=0)])
low_exp = pd.concat([pd.read_csv('low_exp_one_hot_1.csv',index_col=0),
                      pd.read_csv('low_exp_one_hot_2.csv',index_col=0),
                      pd.read_csv('low_exp_one_hot_3.csv',index_col=0),
                      pd.read_csv('low_exp_one_hot_4.csv',index_col=0)])

In [3]:
# concatenate to form a single dataframe
data_df = pd.concat([high_exp, low_exp], axis=0)

#pandas doesn't know how read multiple data types, so they're saved as strings 

# function to convert csv files into 
def string_to_matrix(string):
    # convert string to list of one_hot lists
    string = str(string)
    list_of_strings = string.split('], [')
    list_of_lists = [channels.strip().replace('[', '').replace(']', '').replace(',', '').split() 
                     for channels in list_of_strings
                     if 'nan' not in list_of_strings
                    ]
    # add padding
    remaining_pad = 181 - len(list_of_lists)
    while remaining_pad > 0:
        list_of_lists.append(list([0 for x in range(0, 64)]))
        remaining_pad = remaining_pad - 1
    # return padded one_hot matrix
    return np.array(list_of_lists).astype(np.float)

data_df['one_hot_matrix'] = data_df['one_hot_matrix'].apply(string_to_matrix)

In [4]:
# get X and y data from data_df
max_len = 181
width = 64

X = np.zeros((22615, max_len, width))
for idx, one_hot_matrix in enumerate(data_df['one_hot_matrix'].values):
    X[idx, :, :] = one_hot_matrix

y = data_df['class'].values

# train/test split
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [7]:
# tune hyperparameters for simple model
# define simple model per Yoon Kim (2014)
def create_model(filter_sizes=3, num_filters=10):
    # prepare input shape
    input_shape = (181, 64)
    model_input = Input(shape=input_shape)
    z = model_input

    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(z)
        conv = MaxPooling1D()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    z = Dropout(0.5)(z)
    model_output = Dense(1, activation="sigmoid")(z)

    model = Model(model_input, model_output)
    model.compile(loss="binary_crossentropy", optimizer="adam", 
                  metrics=["accuracy"])

    return model


In [8]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, batch_size=50,
                        epochs=25, verbose=2)

# define the grid search parameters
# model hyperparameters
filter_sizes = [(3, 3, 3), (3, 4, 5), (5, 5, 5),
               (3, 5, 7), (7, 7, 7), (5, 7, 10),
               (10, 10, 10), (3, 4, 5, 6)]
num_filters = [10, 20, 50, 100, 200]


param_grid = dict(filter_sizes=filter_sizes, num_filters=num_filters)

grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    cv=10, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

grid_df = pd.DataFrame(grid_result.cv_results_['params'])
grid_df['means'] = grid_result.cv_results_['mean_test_score']
grid_df['stddev'] = grid_result.cv_results_['std_test_score']

# print results to csv file
grid_df.to_csv('grid_search_results.csv')


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/sw/anaconda-4.3.1/python3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.6/site-packages/ipykernel/__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/sw/anaconda-4.3.1/python

In [ ]:
grid_search_data = pd.read_csv('first_grid_search_results.csv', index_col=0)
grid_search_data.sort_values('means', ascending=False)

In [ ]:
print('Pearson correlation of each hyperparameter \nwith accuracy '
      'from 10-fold cv:')
grid_search_data.corr()['means'][:4]

In [ ]:
grouped = grid_search_data.groupby('filter_size')
grouped.mean()['means'].plot(yerr=grouped.mean()['stddev'], marker='o', ms=7, mfc='k')

In [ ]:
grouped = grid_search_data.groupby(['pool_size', 'filter_size'])
grouped.mean()['means'].plot(yerr=grouped.mean()['stddev'], marker='o', ms=7, mfc='k')

In [ ]:
grouped = grid_search_data.groupby('pool_size')
grouped.mean()['means'].plot(yerr=grouped.mean()['stddev'], marker='o', ms=7, mfc='k')